# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score, classification_report

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sqlalchemy import create_engine


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("InsertTableName", engine)
X = df.message
Y = df.iloc[:,4:]
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    #print("Text:", text)
    tokens = word_tokenize(text)
    lemm = WordNetLemmatizer()
    #print(tokens)
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemm.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok[0])
    #print(clean_tokens)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

accuracy = (y_pred == y_test).mean()
print("Accuracy: ", accuracy)

Accuracy:  related                   0.749313
request                   0.839335
offer                     0.994812
aid_related               0.624046
medical_help              0.917150
medical_products          0.945377
search_and_rescue         0.972536
security                  0.980928
military                  0.968111
child_alone               1.000000
water                     0.938511
food                      0.887702
shelter                   0.905096
clothing                  0.984590
money                     0.978639
missing_people            0.987794
refugees                  0.965670
death                     0.953006
other_aid                 0.860543
infrastructure_related    0.931950
transport                 0.954532
buildings                 0.949039
electricity               0.981385
tools                     0.994355
hospitals                 0.987183
shops                     0.994965
aid_centers               0.986878
other_infrastructure      0.956820
weather_r

In [7]:
y_test.values

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [8]:
y_pred

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [9]:
y_test.values.shape, y_pred.shape

((6554, 36), (6554, 36))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_test_v = y_test.values
y_pred_v = y_pred
score_list = []
for i, col_index in enumerate(y_test.columns.values):
    score = f1_score(y_test_v[i], y_pred_v[i], average='weighted')
    score_list.append(score)
    print(str(col_index) + ": " + str(score))
    
print(np.asarray(score_list).mean())

related: 0.900732600733
request: 0.908955223881
offer: 0.971428571429
aid_related: 1.0
medical_help: 1.0
medical_products: 0.976650563607
search_and_rescue: 1.0
security: 0.985915492958
military: 1.0
child_alone: 0.985915492958
water: 0.985915492958
food: 0.855218855219
shelter: 0.93137254902
clothing: 0.778645833333
money: 1.0
missing_people: 0.872053872054
refugees: 0.900732600733
death: 0.957516339869
other_aid: 1.0
infrastructure_related: 0.855218855219
transport: 0.936026936027
buildings: 1.0
electricity: 0.791397849462
tools: 0.985915492958
hospitals: 0.740740740741
shops: 0.834554334554
aid_centers: 0.971428571429
other_infrastructure: 0.855218855219
weather_related: 0.93137254902
floods: 0.864583333333
storm: 1.0
fire: 0.912698412698
earthquake: 0.971428571429
cold: 0.893532338308
other_weather: 0.740740740741
direct_report: 0.718518518519
0.917067488567


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [11]:
for i, col_index in enumerate(y_test.columns.values):
    print(col_index, classification_report(y_test.values[i], y_pred[i], target_names=col_index))

related              precision    recall  f1-score   support

          r       0.91      1.00      0.95        31
          e       1.00      0.40      0.57         5

avg / total       0.92      0.92      0.90        36

request              precision    recall  f1-score   support

          r       0.94      0.97      0.96        33
          e       0.50      0.33      0.40         3

avg / total       0.90      0.92      0.91        36

offer              precision    recall  f1-score   support

          o       1.00      0.94      0.97        36
          f       0.00      0.00      0.00         0

avg / total       1.00      0.94      0.97        36

aid_related              precision    recall  f1-score   support

          a       1.00      1.00      1.00        36

avg / total       1.00      1.00      1.00        36

medical_help              precision    recall  f1-score   support

          m       1.00      1.00      1.00        36

avg / total       1.00      1.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 7
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 2, does not match size of target_names, 5
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 11
  .format(len(labels), len(target_names))
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 1, does not match size of target_names, 12
  .format(len(labels), len(target_names))
/opt/conda/l

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f2ed8e2a488>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [13]:
parameters = {'vect__max_df': (0.5, 0.75, 1.0),
              'vect__ngram_range': ((1,1), (1,2)),
              'vect__max_features': (None, 1000, 5000),
              'tfidf__use_idf': (True, False)}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_features': (None, 1000, 5000), 'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [14]:
cv.best_params_

{'tfidf__use_idf': True,
 'vect__max_df': 0.5,
 'vect__max_features': None,
 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
y_pred_test = cv.predict(X_test)
y_pred_train = cv.predict(X_train)

In [16]:
y_test_v2 = y_test.values
y_pred_v2 = y_pred_train
score_list = []
for i, col_index in enumerate(y_test.columns.values):
    score = f1_score(y_test_v2[i], y_pred_v2[i], average='weighted')
    score_list.append(score)
    print(str(col_index) + ": " + str(score))
    
print(np.asarray(score_list).mean())

related: 0.938368055556
request: 0.944444444444
offer: 0.941176470588
aid_related: 1.0
medical_help: 0.95652173913
medical_products: 0.957516339869
search_and_rescue: 1.0
security: 1.0
military: 0.976650563607
child_alone: 0.985915492958
water: 0.985915492958
food: 0.866285808909
shelter: 0.869963369963
clothing: 0.718803418803
money: 0.922558922559
missing_people: 0.911355311355
refugees: 0.87360567869
death: 0.905372405372
other_aid: 0.95652173913
infrastructure_related: 0.855218855219
transport: 0.867724867725
buildings: 0.958528951487
electricity: 0.740740740741
tools: 0.95652173913
hospitals: 0.680555555556
shops: 0.855218855219
aid_centers: 0.985915492958
other_infrastructure: 0.812800132472
weather_related: 0.876811594203
floods: 0.789819376026
storm: 0.958528951487
fire: 0.834554334554
earthquake: 0.909090909091
cold: 0.833333333333
other_weather: 0.680555555556
direct_report: 0.666169895678
0.88814067762


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

Due to time constrains in personal life, additional investigation into further improvement, can't me completed at this time.
Hopefully this will not impact the overall scoring and passing the project.

### 9. Export your model as a pickle file

In [17]:
import pickle

In [18]:
filename = 'ML_classifier_data.sv'
file_saved = open(filename, 'wb')
model = pipeline
pickle.dump(model, file_saved)

NameError: name 'model' is not defined

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.